### Base settings

In [ ]:
!pip install openai

In [ ]:
!pip install mysql-connector-python

In [ ]:
# package

import openai
import mysql.connector
from pprint import pprint # chatGPT message가 잘 출력되되도록 사용
import pandas as pd
import numpy as np

In [ ]:
# Google Drive 마운트
# 마운트는 본인 드라이브만 가능하기에 공유 폴더 같은 경우 본인 드라이브에 바로가기를 추가하여 사용

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# API KEY : contact 계정의 API key 발급하여 사용

openai.api_key = ""

In [ ]:
# google sheet를 바로 연동하는 것은 원본 훼손손 위험이 존재하기에 우선 xlsx 파일을 다운로드 및 업로드하여여 활용

raw_data = pd.read_excel("/content/drive/MyDrive/HAI 연구개발 리서치/data/PoC 평가.xlsx", sheet_name='total')

### Input 불러오기

In [ ]:
en_data = raw_data.copy()

In [ ]:
# PoC 평가가 A+인 인원들만 불러오기
# url 주소에 userID가 존재하는 형식 데이터만 불러오기
en_data = en_data.loc[en_data['총평 \n(Letter grade, like A+, B+)'] == 'A+']
en_data = en_data[en_data['framing docs'].str.contains('&userID', na = False)].reset_index(drop=True)

In [ ]:
# '&' 기준으로 파싱하여 user_id 리스트에 저장하기
id_list = []
for i in range(len(en_data)) :
  id_list.append(en_data['framing docs'][i].split('&')[1][7:])

In [ ]:
# DB 연동

mydb = mysql.connector.connect(
  host="",
  user="",
  password="",
  database=""
)

# 저장된 userID 정보 불러와서 사용
userCode = id_list[0]

mycursor = mydb.cursor()

mycursor.execute("SELECT user_id from userCodeTable where code='" + userCode + "'")

myresult = mycursor.fetchall()

userID = myresult[0][0]

mycursor.execute("SELECT user_id, question_id, response FROM responseTable where user_id='" + str(userID) + "'")

myresult = mycursor.fetchall()

userResponse = {}

# 0부터 시작되는 문항번호를 1부터 시작하도록 변경
for x in myresult:
    userResponse[x[1]+1] = x[2]

In [ ]:
# PoC 데이터는 dictionary 형태로 저장
userResponse

In [ ]:
# 문항 정보 입력
qna = pd.read_csv("/content/drive/MyDrive/HAI 연구개발 리서치/data/PoC_Question.csv")

### chatGPT 활용하여 Output 산출 (1차 trial and error)


*   29번 문항까지 메시지 응답을 주고 받을 때, 토큰 초과 이슈가 발생하여 특정 문항 중심으로 선별해서 질문할 필요성이 있음



In [ ]:
# 초기 system role 세팅
messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"}]

In [ ]:
# 29번 문항까지 메시지 응답을 주고 받을 때, 토큰 초과 이슈가 발생하여 특정 문항 중심으로 선별해서 질문할 필요성이 있음
for i in range(len(userResponse)) :
  messages.append({"role" : "assistant", "content" : f"{str(qna['문항번호'][i])}번 문항 : {qna['문항'][i]}"})
  messages.append({"role" : "user", "content" : f"{i+1}번 문항 답변 : {userResponse[i+1]}"})

  if i >= 3 :
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=1
    )

    chat_response = completion.choices[0].message.content
    messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
# output 결과 저장
output = pd.DataFrame.from_dict(data=messages, orient='columns')
output.to_csv(f"/content/drive/MyDrive/HAI 연구개발 리서치/output/230331_output_{id_list[0]}.csv")

### chatGPT 활용하여 Output 산출 (2차 trial and error)


*   16개 문항을 제거하였지만 chatGPT의 답변이 더욱 구체적으로 도출되었는지 오히려 더 많은 문항을 한번에 다루지 못함(14번까지)
*   이전 output 비교했을 때, 전자가 54, 후자가 27개 index를 가지고 있어 문항은 줄었지만 output도 절반으로 줄음
*   핵심 문항으로 더 축소할 필요 있음



In [ ]:
# 일부 문항 제거하여 dictionary 구성
userResponse_input = userResponse.copy()
drop_user_list = [1, 2, 3, 11, 12, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27]
[userResponse_input.pop(key, None) for key in drop_user_list]
userResponse_input

In [ ]:
# 메시지 초기화 및 system role 부여
messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"}]

In [ ]:
# 문항을 줄였지만 여전히 토큰 초과 오류 발생
for i in userResponse_input.keys() :
  messages.append({"role" : "assistant", "content" : f"{str(qna['문항번호'][i-1])}번 문항 : {qna['문항'][i-1]}"})
  messages.append({"role" : "user", "content" : f"{i}번 문항 답변 : {userResponse_input[i]}"})

  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=1
  )

  chat_response = completion.choices[0].message.content
  messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
# output 결과 저장
output_2 = pd.DataFrame.from_dict(data=messages, orient='columns')
output_2.to_csv(f"/content/drive/MyDrive/HAI 연구개발 리서치/output/230331_output_2_{id_list[0]}.csv")

### chatGPT 활용하여 Output 산출 (3차 trial and error)


*   6개 문항으로 축소하였고 error 발생하지 않음
*   토큰 수의 한계가 있기 때문에 현재 별다른 프롬프트 엔지니어링을 하지 않은 상태이기에 적절하게 user의 질문 변형 필요 있음
*   교육생이 많은 텍스트를 작성할 때에 대한 방안도 고민 필요
*   assistant로 문항 정보를 가져왔는데 불필요한 토큰을 늘리는 것이 아닌지에 대해서도 고민 필요



In [ ]:
# 메시지 초기화 및 system role 부여
messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"}]

In [ ]:
# 6개 문항 선택
for i in [5, 6, 7, 8, 9, 10] :
  messages.append({"role" : "assistant", "content" : f"{str(qna['문항번호'][i-1])}번 문항 : {qna['문항'][i-1]}"})
  messages.append({"role" : "user", "content" : f"{i}번 문항 답변 : {userResponse[i]}"})

  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=1
  )

  chat_response = completion.choices[0].message.content
  messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
output_3 = pd.DataFrame.from_dict(data=messages, orient='columns')
output_3.to_csv(f"/content/drive/MyDrive/HAI 연구개발 리서치/output/230331_output_3_{id_list[0]}.csv")

In [ ]:
output_3

In [ ]:
# 메시지 초기화 및 system role 부여
messages = [{"role": "system", "content": "너는 AI 분야의 data scientist야"}]

In [ ]:
# 6개 문항 선택
for i in [5, 8, 22] :
  messages.append({"role" : "assistant", "content" : f"{str(qna['문항번호'][i-1])}번 문항 : {qna['문항'][i-1]}"})
  messages.append({"role" : "user", "content" : f"{i}번 문항 답변 : {userResponse[i]}"})

  completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  temperature=1
  )

  chat_response = completion.choices[0].message.content
  messages.append({"role" : "assistant", "content" : chat_response})

In [ ]:
output_4 = pd.DataFrame.from_dict(data=messages, orient='columns')
output_4.to_csv(f"/content/drive/MyDrive/HAI 연구개발 리서치/output/230331_output_4_{id_list[0]}.csv")

In [ ]:
output_4